In [1]:
from flame.flame_model_creation import FlameModelCreation
from data_processing.dataset import FaceDataset
from gaussian_splatting.utils.graphics_utils import BasicPointCloud
import torch
import k3d
import yaml
import numpy as np
import os
from gaussian_splatting.utils.sh_utils import SH2RGB

os.environ["DISPLAY"] = ":10.0"

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
face_dataset = FaceDataset(yaml.safe_load(open('./configs/settings.yaml', 'r')))

def get_aligned_pointcloud(flame_params_path):
    vertices = FlameModelCreation.create_flame_model(flame_params_path, upsample_flame_iterations=1)
    pointlcoud = BasicPointCloud(vertices, torch.randn((len(vertices), 3)), None)
    
    #new_pointcloud = FaceDataset.normalize_flame_and_cameras(None, pointlcoud, None)
    
    return pointlcoud

def get_images(flame_params_path, camera_params):
    image_path = flame_params_path[:flame_params_path.find('/annotations/')]
    image_path += f"/timesteps/frame_00000/images-2x"
    return face_dataset.get_images({'images_path': image_path}, camera_params)

#def get_farl_features(flame_params_path, )

def get_translated_camera_params(flame_params_path):
    camera_params_path = flame_params_path[:flame_params_path.find('/sequences/')]
    camera_params_path += f"/calibration/camera_params.json"
    camera_params = face_dataset.get_camera_params({'camera_params_path': camera_params_path})
    face_dataset.scale_and_rotate_camera_params(camera_params, {'flame_params_path': flame_params_path})
    
    return camera_params

def get_colored_gaussians(flame_params_path, pointcloud, camera_params, images):
    image_path = flame_params_path[:flame_params_path.find('/annotations/')]
    image_path += f"/timesteps/frame_00000/gaussian_colors.npy"
    input_serials = ["222200036", "222200049"]
    return face_dataset.get_projected_colors({'gaussian_colors_path': image_path}, camera_params, pointcloud, {serial: image for serial, image in images.items() if serial in input_serials})


def get_farl_gaussians(flame_params_path, pointcloud, camera_params, images):
    farl_path = flame_params_path[:flame_params_path.find('/annotations/')]
    farl_path += f"/timesteps/frame_00000/gaussian_features.npy"
    
    image_path = flame_params_path[:flame_params_path.find('/annotations/')]
    image_path += f"/timesteps/frame_00000/images-2x"
    input_serials = ["221501007"]
    return face_dataset.get_projected_farl_features({'gaussian_features_path': farl_path, 'images_path': image_path}, camera_params, pointcloud, {serial: image for serial, image in images.items() if serial in input_serials})

def do_all(path):
    p1_pointcloud = get_aligned_pointcloud(path)
    p1_camera_params = get_translated_camera_params(path)
    p1_images = get_images(path, p1_camera_params)
    colors = SH2RGB(get_colored_gaussians(path, np.array(p1_pointcloud.points), p1_camera_params, p1_images))*255
    farl = get_farl_gaussians(path, np.array(p1_pointcloud.points), p1_camera_params, p1_images)
    
    return p1_pointcloud, colors, farl

train self.face_ids ['017', '018', '024', '030', '031', '032', '033', '035', '036', '037', '038', '040', '042', '043', '055', '056', '057', '058', '060', '061', '063', '064', '065', '067', '068', '068', '069', '071', '072', '073', '074', '075', '076', '077', '078', '079', '080', '081', '082', '083', '085', '088', '089', '090', '091', '092', '093', '094', '095', '096', '097', '098', '099', '100', '102', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', '151', '162', '163', '164', '165', '166', '167', '168', '170', '171', '172', '173', '174', '175', '176', '177', '178', '179', '180', '181', '182', '183', '184', '185', '186', '187', '188', '189', '191', '192', '193', '194', '195', '197', '198', '199', '200', '201', '202', '203',

In [3]:
p1_path = '/home/oroz/gaussian-splatting/3DSSL-SS24_Generalizable3DHeadReconstruction/data/032/sequences/SEN-10-port_strong_smokey/annotations/tracking/FLAME2023_v2/tracked_flame_params.npz'
p2_path = '/home/oroz/gaussian-splatting/3DSSL-SS24_Generalizable3DHeadReconstruction/data/032/sequences/SEN-10-port_strong_smokey/annotations/tracking/FLAME2023_v2/tracked_flame_params.npz'
p3_path = '/home/oroz/gaussian-splatting/3DSSL-SS24_Generalizable3DHeadReconstruction/data/032/sequences/SEN-10-port_strong_smokey/annotations/tracking/FLAME2023_v2/tracked_flame_params.npz'
p4_path = '/home/oroz/gaussian-splatting/3DSSL-SS24_Generalizable3DHeadReconstruction/data/032/sequences/SEN-10-port_strong_smokey/annotations/tracking/FLAME2023_v2/tracked_flame_params.npz'

p1_pointcloud, colors, farl = do_all(p1_path)

plot = k3d.plot()

color_map = [((int(r) << 16) | (int(g) << 8) | int(b)) for r, g, b in colors.tolist()]

plot += k3d.points(p1_pointcloud.points, point_size=0.0025, colors=color_map)
#plot += k3d.points(p2_pointcloud.points, point_size=0.01, color=0x00ff00)
#plot += k3d.points(p3_pointcloud.points, point_size=0.01, color=0x0000ff)
#plot += k3d.points(p4_pointcloud.points, point_size=0.01, color=0x000000)

plot.display()

TypeError: create_flame_model() got an unexpected keyword argument 'upsample_flame_iterations'

In [ ]:
plot = k3d.plot()

def interpolate_color(value):
    start_color = [0, 0, 0]
    end_color = [0, 255, 0]
    r = start_color[0] + (end_color[0] - start_color[0]) * value
    g = start_color[1] + (end_color[1] - start_color[1]) * value
    b = start_color[2] + (end_color[2] - start_color[2]) * value
    return int(r) << 16 | int(g) << 8 | int(b)

color_map = [interpolate_color(gaussian[769]) for gaussian in farl.tolist()]

plot += k3d.points(p1_pointcloud.points, point_size=0.01, colors=color_map)

plot.display()

Output()